In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
data.head(5)

,Product_ID,Category,Supplier,Warehouse_Location,Order_Date,Delivery_Date,Inventory_Level,Stockouts,Unit_Cost,Lead_Time
0,P0001,Toys,Supplier B,Chicago,2025-02-23,2025-03-01,15,0,446.20,6
1,P0002,Furniture,Supplier C,San Francisco,2025-02-25,2025-03-15,138,0,208.02,18
2,P0003,Furniture,Supplier B,Chicago,2025-01-22,2025-02-08,270,0,423.75,14
3,P0004,Toys,Supplier D,Chicago,2025-01-29,2025-02-06,151,0,123.17,8
4,P0005,Food,Supplier A,Dallas,2025-03-10,2025-03-24,237,1,462.93,14


In [4]:
data.shape

(250, 10)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product_ID          250 non-null    object 
 1   Category            250 non-null    object 
 2   Supplier            250 non-null    object 
 3   Warehouse_Location  250 non-null    object 
 4   Order_Date          250 non-null    object 
 5   Delivery_Date       250 non-null    object 
 6   Inventory_Level     250 non-null    int64  
 7   Stockouts           250 non-null    int64  
 8   Unit_Cost           250 non-null    float64
 9   Lead_Time           250 non-null    int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 19.7+ KB


In [6]:
data.describe()

,Inventory_Level,Stockouts,Unit_Cost,Lead_Time
count,250.000000,250.000000,250.000000,250.0000
mean,256.348000,0.188000,247.996080,10.0000
std,138.852392,0.391496,138.993919,5.0785
min,1.000000,0.000000,5.570000,1.0000
25%,149.000000,0.000000,126.965000,6.0000
50%,268.000000,0.000000,247.265000,10.0000
75%,373.750000,0.000000,377.477500,14.0000
max,499.000000,1.000000,493.750000,19.0000


In [7]:
data.isnull().sum()

Product_ID            0
Category              0
Supplier              0
Warehouse_Location    0
Order_Date            0
Delivery_Date         0
Inventory_Level       0
Stockouts             0
Unit_Cost             0
Lead_Time             0
dtype: int64

In [8]:
data.duplicated().sum()

np.int64(0)

In [9]:
data.value_counts()

Product_ID  Category     Supplier    Warehouse_Location  Order_Date  Delivery_Date  Inventory_Level  Stockouts  Unit_Cost  Lead_Time
P0001       Toys         Supplier B  Chicago             2025-02-23  2025-03-01     15               0          446.20     6            1
P0002       Furniture    Supplier C  San Francisco       2025-02-25  2025-03-15     138              0          208.02     18           1
P0003       Furniture    Supplier B  Chicago             2025-01-22  2025-02-08     270              0          423.75     14           1
P0004       Toys         Supplier D  Chicago             2025-01-29  2025-02-06     151              0          123.17     8            1
P0005       Food         Supplier A  Dallas              2025-03-10  2025-03-24     237              1          462.93     14           1
                                                                                                                                       ..
P0246       Toys         Supplier C  Ch

In [10]:
data.nunique()

Product_ID            250
Category                5
Supplier                4
Warehouse_Location      4
Order_Date             79
Delivery_Date          80
Inventory_Level       200
Stockouts               2
Unit_Cost             250
Lead_Time              19
dtype: int64

In [11]:
data_supply_lead = data.groupby('Supplier')['Lead_Time'].mean()
data_supply_lead

Supplier
Supplier A    10.672414
Supplier B     9.015385
Supplier C     9.600000
Supplier D    10.822581
Name: Lead_Time, dtype: float64